In [39]:
import pandas as pd
import numpy as np

# Retrieve TYNDP data

In [46]:
etm_demand = pd.read_excel('Demand_Scenarios_TYNDP_2024_After_Public_Consultation.xlsb', sheet_name='3_DEMAND_OUTPUT', header=1)
etm_demand = etm_demand.copy()
etm_demand.drop(columns=["STUDY", "TYPE", "DASHBOARD_ID", "PARAMETER", "ENERGY_TYPE", "UNIT"], inplace=True)
print(f"Sectors available:\n{etm_demand['SECTOR'].unique()}")
print(f"Subsectors available:\n{etm_demand['SUBSECTOR'].unique()}")
etm_demand_elc = etm_demand[(etm_demand['ENERGY_CARRIER'] == 'Electricity')
                            & (etm_demand['COUNTRY'] == 'IT')
                            ].drop(columns=['COUNTRY', 'ENERGY_CARRIER'])
etm_demand_elc = etm_demand_elc.reset_index(drop=True).set_index(['SECTOR', 'SUBSECTOR'])

print("============> Electricity consumption by sector - TYNDP (TWh)")
display(round(etm_demand_elc,2))


heating_index = ['Space heating', 'Space heating & hot water']
hot_water_index = ['Hot water']
cooling_index = ['Cooling']
etm_demand_heat_cool = etm_demand[(etm_demand['SECTOR'].isin(["Households", "Buildings"]))
                             & (etm_demand['SUBSECTOR'].isin(heating_index + cooling_index))
                            & (etm_demand['COUNTRY'] == 'IT')
                            ].drop(columns=['COUNTRY', "SECTOR", "SUBSECTOR"]).groupby(["ENERGY_CARRIER"]).sum()
print("============> Heating and cooling demand by carrier - TYNDP (TWh)")
display(round(etm_demand_heat_cool,2))

Sectors available:
['Households' 'Buildings' 'Industry' 'Transport' 'Agriculture' 'Other'
 'Energy']
Subsectors available:
['Total' 'Space heating' 'Cooling' 'Hot water' 'Appliances'
 'Space heating & hot water' 'Aluminium' 'Chemicals' 'Fertilizers' 'Food'
 'Metals' 'Others' 'Paper' 'Refineries' 'Steel' 'Cars' 'Busses' 'Trucks'
 'Vans' 'Passenger trains' 'Freight trains' 'Ships' 'Planes'
 'International aviation' 'International shipping' 'Datacenters']
============> Electricity consumption by sector - TYNDP (TWh)


2019    2040    2050  2040.1  2050.1
SECTOR      SUBSECTOR                                                        
Households  Total                       63.16   90.17   93.15   83.96   87.30
Buildings   Total                       82.32   79.04   65.69   77.82   65.52
Industry    Total                      136.44  153.81  172.14  146.99  158.37
            Total                        0.00    0.00    0.00    0.00    0.00
Transport   Total                        5.90   60.10   96.67   50.24   82.10
Agriculture Total                        5.84    7.19    8.50    6.01    6.01
Other       Total                        0.00    0.00    0.00    0.00    0.00
Energy      Total                        9.90    9.90    9.90    9.90    9.90
Households  Space heating                4.66   24.69   29.63   22.02   27.30
            Cooling                      7.99    9.18   10.20    9.40   10.51
            Hot water                    2.00   11.42   15.24    8.39   12.50
            Appliances                  48.52   44.89   38.08   44.16   36.99
Buildings   Space heating & hot water   19.97   19.01   11.28   17.70   11.09
            Cooling                     10.42    8.34    6.78    8.43    6.80
            Appliances                  51.93   51.69   47.63   51.69   47.63
Industry    Aluminium                    0.00    0.00    0.00    0.00    0.00
            Chemicals                   13.55   13.99   16.38   12.37   11.14
            Chemicals                    0.00    0.00    0.00    0.00    0.00
            Fertilizers                  0.11    0.10    0.09    0.09    0.07
            Fertilizers                  0.00    0.00    0.00    0.00    0.00
            Food                        13.46   13.17   14.05   11.09   10.49
            Metals                       2.04    1.39    1.16    1.39    1.16
            Others                      66.58   78.68   90.20   74.07   83.93
            Others                       0.00    0.00    0.00    0.00    0.00
            Paper                        8.70    7.58    7.94    6.49    6.27
            Refineries                   5.99    2.03    0.84    2.52    1.12
            Refineries                   0.00    0.00    0.00    0.00    0.00
            Steel                       18.98   19.83   20.45   21.92   23.16
Transport   Cars                         0.05   39.53   60.63   35.22   56.01
            Busses                       0.04    2.27    6.22    1.69    4.08
            Trucks                       0.00    4.14   10.35    2.61    5.82
            Vans                         0.01    7.09   11.12    5.08    9.93
            Passenger trains             4.68    4.30    4.15    4.28    4.15
            Freight trains               0.87    0.71    0.70    0.71    0.70
            Ships                        0.21    0.20    0.22    0.20    0.22
            Planes                       0.00    1.45    2.45    0.24    0.40
            Others                       0.04    0.41    0.85    0.20    0.79
            International aviation       0.00    0.00    0.00    0.00    0.00
            International shipping       0.00    0.00    0.00    0.00    0.00
Industry    Datacenters                  7.03   17.04   21.03   17.04   21.03

============> Heating and cooling demand by carrier - TYNDP (TWh)


,2019,2040,2050,2040.1,2050.1
ENERGY_CARRIER,,,,,
Biofuels,60.40,35.51,17.54,35.57,23.33
Electricity,43.03,61.21,57.89,57.55,55.70
Heat,12.89,12.10,11.79,10.03,8.57
Hydrogen,0.00,1.56,7.85,10.98,16.19
Liquids,22.18,0.00,0.00,0.00,0.00
Methane,178.41,65.26,26.86,75.49,30.19
Others,0.00,0.00,0.00,0.00,0.00
Solids,0.00,0.00,0.00,0.00,0.00


# 1 - Postprocess data: electricity demand by sector

In [ ]:
# Total
etm_demand_elc_tot = etm_demand_elc[etm_demand_elc.index.get_level_values('SUBSECTOR') == 'Total']
total = etm_demand_elc_tot.sum().to_frame().rename(columns={0: 'Total'})

# Transport
etm_demand_elc_tra = etm_demand_elc[
    (etm_demand_elc.index.get_level_values('SECTOR') == 'Transport') &
    (etm_demand_elc.index.get_level_values('SUBSECTOR') == 'Total')
    ]
transport = etm_demand_elc_tra.sum().to_frame().rename(columns={0: 'Transport'})

# Households and Buildings: heating
etm_demand_elc_heat = etm_demand_elc[
    ((etm_demand_elc.index.get_level_values('SECTOR') == 'Households') |
    (etm_demand_elc.index.get_level_values('SECTOR') == 'Buildings')) &
    (etm_demand_elc.index.get_level_values('SUBSECTOR').isin(heating_index))
    ]
heating = etm_demand_elc_heat.sum().to_frame().rename(columns={0: 'Heating'})

# Households and Buildings: hot water
etm_demand_elc_hotwater = etm_demand_elc[
    ((etm_demand_elc.index.get_level_values('SECTOR') == 'Households') |
    (etm_demand_elc.index.get_level_values('SECTOR') == 'Buildings')) &
    (etm_demand_elc.index.get_level_values('SUBSECTOR').isin(hot_water_index))
    ]
hot_water = etm_demand_elc_hotwater.sum().to_frame().rename(columns={0: 'Hot Water'})

# Households and Buildings: cooling
etm_demand_elc_cool = etm_demand_elc[
    ((etm_demand_elc.index.get_level_values('SECTOR') == 'Households') |
    (etm_demand_elc.index.get_level_values('SECTOR') == 'Buildings')) &
    (etm_demand_elc.index.get_level_values('SUBSECTOR').isin(cooling_index))
    ]
cooling = etm_demand_elc_cool.sum().to_frame().rename(columns={0: 'Cooling'})

## All scenarios

In [42]:
df_abs = round(pd.concat([total,
                          transport,
                          heating,
                          hot_water,
                          cooling], axis=1).rename(
                              index = {2040: 'DS-2040',
                                       2050: 'DS-2050',
                                       '2040.1': 'GA-2040',
                                       '2050.1': 'GA-2050'
                                       }),0)

# DataFrame con gli share rispetto al totale (in percentuale)
df_share = round(df_abs.div(df_abs['Total'], axis=0) * 100, 0).drop(columns=['Total'])

# DataFrame con gli aumenti percentuali rispetto al 2019
df_growth = round((df_abs / df_abs.loc[2019] - 1) * 100, 0).drop(index=[2019])


## Average scenario

In [43]:
def df_avg(df, reference=True):
    if reference:
        df_avg = pd.DataFrame({
        2019: df.loc[2019],
        2040: df.loc[['DS-2040', 'GA-2040']].mean(),
        2050: df.loc[['DS-2050', 'GA-2050']].mean()
    }).T.round(0)
    else:
        df_avg = pd.DataFrame({
        2040: df.loc[['DS-2040', 'GA-2040']].mean(),
        2050: df.loc[['DS-2050', 'GA-2050']].mean()
    }).T.round(0)
    return df_avg

print("====================> Final elc consumption - average scenario (TWh)")
display(df_avg(df_abs))

# Share rispetto al totale
print("====================> Final elc consumption - share over total (%)")
display(df_avg(df_share))

# Crescita rispetto al 2019
print("====================> Final elc consumption - growth over 2019 (%)")
display(df_avg(df_growth, reference=False))

====================> Final elc consumption - average scenario (TWh)


,Total,Transport,Heating,Hot Water,Cooling
2019,304.0,6.0,25.0,2.0,18.0
2040,388.0,55.0,42.0,10.0,18.0
2050,428.0,90.0,40.0,14.0,17.0


====================> Final elc consumption - share over total (%)


,Transport,Heating,Hot Water,Cooling
2019,2.0,8.0,1.0,6.0
2040,14.0,11.0,2.0,4.0
2050,21.0,9.0,3.0,4.0


====================> Final elc consumption - growth over 2019 (%)


,Total,Transport,Heating,Hot Water,Cooling
2040,28.0,816.0,68.0,375.0,0.0
2050,41.0,1392.0,58.0,575.0,-6.0


# 2 - Postprocess data: heating and cooling demand by carrier

## All scenarios

In [44]:
# Transporre per avere i carrier come colonne
df_abs_carriers = round(etm_demand_heat_cool.T.rename(
    index = {2040: 'DS-2040',
             2050: 'DS-2050',
             '2040.1': 'GA-2040',
             '2050.1': 'GA-2050'
             }),0)

# Calcolare il totale per ogni scenario
df_abs_carriers['Total'] = df_abs_carriers.sum(axis=1)

# DataFrame con gli share rispetto al totale (in percentuale)
df_share_carriers = round(df_abs_carriers.drop(columns=['Total']).div(df_abs_carriers['Total'], axis=0) * 100, 0)

# DataFrame con gli aumenti percentuali rispetto al 2019
df_growth_carriers = round((df_abs_carriers / df_abs_carriers.loc[2019] - 1) * 100, 0).drop(index=[2019])

## Average scenario

In [45]:
print("====================> Heating and cooling demand by carrier - average scenario (TWh)")
display(df_avg(df_abs_carriers))

# Share rispetto al totale
print("====================> Heating and cooling demand by carrier - share over total (%)")
display(df_avg(df_share_carriers))

# Crescita rispetto al 2019
print("====================> Heating and cooling demand by carrier - growth over 2019 (%)")
display(df_avg(df_growth_carriers, reference=False))

====================> Heating and cooling demand by carrier - average scenario (TWh)


ENERGY_CARRIER,Biofuels,Electricity,Heat,Hydrogen,Liquids,Methane,Others,Solids,Total
2019,60.0,43.0,13.0,0.0,22.0,178.0,0.0,0.0,316.0
2040,36.0,60.0,11.0,6.0,0.0,70.0,0.0,0.0,183.0
2050,20.0,57.0,10.0,12.0,0.0,28.0,0.0,0.0,128.0


====================> Heating and cooling demand by carrier - share over total (%)


ENERGY_CARRIER,Biofuels,Electricity,Heat,Hydrogen,Liquids,Methane,Others,Solids
2019,19.0,14.0,4.0,0.0,7.0,56.0,0.0,0.0
2040,20.0,33.0,6.0,4.0,0.0,38.0,0.0,0.0
2050,16.0,44.0,8.0,10.0,0.0,22.0,0.0,0.0


====================> Heating and cooling demand by carrier - growth over 2019 (%)


ENERGY_CARRIER,Biofuels,Electricity,Heat,Hydrogen,Liquids,Methane,Others,Solids,Total
2040,-40.0,38.0,-16.0,inf,-100.0,-60.0,NaN,NaN,-42.0
2050,-66.0,32.0,-20.0,inf,-100.0,-84.0,NaN,NaN,-60.0
